### Demo plain midi with Sakamoto

In [1]:
import mido
import sys
import binascii
import pprint
from collections import OrderedDict
from mido import Message, MidiFile, MidiTrack
from enum import Enum
from copy import copy, deepcopy

In [2]:
### Need to deal with variable length -- for both "len" of meta-event, and "delta-time"
### Need to deal with running status -- http://midi.teragonaudio.com/tech/midispec/run.htm

MSB_1 = ['8', '9', 'a', 'b', 'c', 'd', 'e', 'f']

## TODO: ff00
## TODO: ff20, 21
## TODO: ff7f
## TODO: running status for all events

END_OF_TRACK = 'ff2f00'
TEXT_META_EVENTS = [f'ff0{i+1}' for i in range(9)]
TEMPO = 'ff5103'
SMPTE = 'ff5405'
TIME_SIG = 'ff5804'
KEY_SIG = 'ff5902'

class Track:
    def __init__(self, track):
        self.track = track
        self._parse()
    
    def _get_varlen_value (self, string):
        value = ''
        j = 0
        while True:
            if string[j] not in MSB_1: # reached last byte of variable-length field
                value = string[0:j+2]
                break
            else:
                j += 2
        return value
        
    def _varFieldToInt(self, hexstring):
        ret = 0
        current_byte = ''

        i=0
        while len(hexstring):
            current_byte = "0x" + hexstring[0:2]
            current_integer = int(current_byte, 16)
            ret = (ret << 7) | (current_integer & 0x7f)
            if not current_integer & 0x80:
                return ret
            hexstring = hexstring[2:]

            i += 1
            if i>1000:
                print('error')
                break
    
    def _parse(self):
        D = []
        
        assert self.track[0:8] == '4d54726b'
        self.chunklen = self.track[8:8+8]
        D.append({'type':'header', 'value':'4d54726b'})
        D.append({'type':'chunklen', 'value':self.track[8:8+8]})
        
        track = self.track[16:]
        i = 0
        state = 'delta-time' # state in ['delta-time', 'event']
        while True:
            if state == 'delta-time':
                value = self._get_varlen_value(track[i:])
                D.append( {'type':'delta time', 'value':value} )
                i += len(value)
                state = 'event'
                continue
                
            else: # state == 'event'
                running = False
                ## first decode event
                if track[i] not in MSB_1: # running status in use
                    event = last_event # for note on / note off / control change / program change - include channel n
                    running = True
                else:
                    track_first6 = track[i:i+6]
                    track_first4 = track[i:i+4]
                    track_first = track[i]
                    if track_first6 == END_OF_TRACK: # end of track; ff 2f 00
                        event = END_OF_TRACK
                    elif track_first6 == TIME_SIG: # ff 58 04 nn dd cc bb
                        event = TIME_SIG
                    elif track_first6 == KEY_SIG: # ff 59 02 sf mi
                        event = KEY_SIG
                    elif track_first6 == TEMPO: # ff 51 03 tt tt tt
                        event = TEMPO
                    elif track_first6 == SMPTE: # ff 54 05 hr mn se fr ff
                        event = SMPTE
                    elif track_first4 in TEXT_META_EVENTS:
                        event = track_first4
                    elif track_first in ['b','c','8','9']:
                        event = track[i:i+2]
                    
                ## then parse event and append to event list
                if event == END_OF_TRACK: # end of track; ff 2f 00
                    D.append( {'type':'end of track', 'type hex':END_OF_TRACK, 'value':''} )
                    break
                
                elif event == TIME_SIG: # ff 58 04 nn dd cc bb
                    D.append( {'type':'time signature', 'type hex':track_first6, 'value':track[i+6:i+6+8]} )
                    i += 14
                    
                elif event == KEY_SIG: # ff 59 02 sf mi
                    D.append( {'type':'key signature', 'type hex':track_first6, 'value':track[i+6:i+6+4]} )
                    i += 10
                
                elif event == TEMPO: # ff 51 03 tt tt tt
                    D.append( {'type':'tempo', 'type hex':track_first6, 'value':track[i+6:i+6+6]} )
                    i += 12
                
                elif event == SMPTE: # ff 54 05 hr mn se fr ff
                    D.append( {'type':'smpte', 'type hex':track_first6, 'value':track[i+6:i+6+10]} )
                    i += 16
                    
                # sequence/track name (ff 03 length text)
                # or marker (ff 06 length text)
                # or cue (ff 07 length text)
                elif event in TEXT_META_EVENTS:
                    if running:
                        name_length_varlen = self._get_varlen_value(track)
                        name_length = self._varFieldToInt(name_length_varlen)
                        name = track[i + len(name_length_varlen) : i + len(name_length_varlen) + name_length*2]
                        D.append( {'type':'text meta event', 'type hex':event, 'omitted':2, ## omitted 2 bytes
                                   'length_varlen':name_length_varlen, 'name':name} )
                        i += len(name_length_varlen) + name_length*2 # name_length * byte, every byte == 2 hex
                    else:
                        name_length_varlen = self._get_varlen_value(track[i+4:])
                        name_length = self._varFieldToInt(name_length_varlen)
                        name = track[i + 4 + len(name_length_varlen) : i + 4 + len(name_length_varlen) + name_length*2]
                        D.append( {'type':'text meta event', 'type hex':track_first4, 'omitted':0,
                                   'length_varlen':name_length_varlen, 'name':name} )
                        i += 4 + len(name_length_varlen) + name_length*2 # name_length * byte, every byte == 2 hex
                
                # Assuming only bn/cn/8n/9n can use running-status
                elif event[0] == 'b': # control change; bn controller value
                    if running:
                        D.append( {'type':'control change', 'type hex':'b', 'omitted':1,
                                   'channel':event[1], 'controller':track[i:i+2], 'value':track[i+2:i+4]} )
                        i += 4
                    else:
                        D.append( {'type':'control change', 'type hex':'b', 'omitted':0,
                                   'channel':track[i+1], 'controller':track[i+2:i+4], 'value':track[i+4:i+6]} )
                        i += 6
                
                elif event[0] == 'c': # program change; cn program
                    if running:
                        D.append( {'type':'program change', 'type hex':'c', 'omitted':1,
                                   'channel':event[1], 'program':track[i:i+2]} )
                        i += 2
                    else:
                        D.append( {'type':'program change', 'type hex':'c', 'omitted':0,
                                   'channel':track[i+1], 'program':track[i+2:i+4]} )
                        i += 4
                
                elif event[0] == '9': # note on; 9n note velocity
                    if running:
                        D.append( {'type':'note on', 'type hex':'9', 'omitted':1,
                                   'channel':event[1], 'note':track[i:i+2], 'velocity':track[i+2:i+4]} )
                        i += 4
                    else:
                        D.append( {'type':'note on', 'type hex':'9', 'omitted':0,
                                   'channel':track[i+1], 'note':track[i+2:i+4], 'velocity':track[i+4:i+6]} )
                        i += 6
                    
                elif track_first == '8': # note off; 8n note velocity
                    if running:
                        D.append( {'type':'note off', 'type hex':'8', 'omitted':1,
                                   'channel':event[1], 'note':track[i:i+2], 'velocity':track[i+2:i+4]} )
                        i += 4
                    else:
                        D.append( {'type':'note off', 'type hex':'8', 'omitted':0,
                                   'channel':track[i+1], 'note':track[i+2:i+4], 'velocity':track[i+4:i+6]} )
                        i += 6
                
                last_event = event
                state = 'delta-time'
                
        self.segs = D    
    
    def segs_to_hexstring(self): # segs is list of events, each event is a dictionary
        hexstring = ''

        for event in self.segs:
            hexstring += Track.event_to_hexstring(event)
        return hexstring

    @staticmethod
    def event_to_hexstring(event):
        if event['type'] == 'header' or event['type'] == 'chunklen' or event['type'] == 'delta time':
            hexstring = event['value']

        elif event['type'] == 'end of track':
            hexstring = event['type hex']

        elif event['type'] == 'text meta event':
            hexstring = event['type hex']*(not event['omitted']) + event['length_varlen'] + event['name']

        elif event['type'] in ['time signature', 'key signature', 'tempo', 'smpte']:
            hexstring = event['type hex'] + event['value']

        elif event['type'] == 'control change':
            hexstring = (event['type hex']+event['channel'])*(not event['omitted']) + event['controller'] + event['value']

        elif event['type'] == 'program change':
            hexstring = (event['type hex']+event['channel'])*(not event['omitted']) + event['program']

        elif event['type'] == 'note on' or event['type'] == 'note off':
            hexstring = (event['type hex']+event['channel'])*(not event['omitted']) + event['note'] + event['velocity']

        return hexstring
    
    @staticmethod
    def event_to_hexstring_discard_runningstatus(event):
        if event['type'] == 'header' or event['type'] == 'chunklen' or event['type'] == 'delta time':
            hexstring = event['value']

        elif event['type'] == 'end of track':
            hexstring = event['type hex']

        elif event['type'] == 'text meta event':
            hexstring = event['type hex'] + event['length_varlen'] + event['name']

        elif event['type'] in ['time signature', 'key signature', 'tempo', 'smpte']:
            hexstring = event['type hex'] + event['value']

        elif event['type'] == 'control change':
            hexstring = event['type hex'] + event['channel'] + event['controller'] + event['value']

        elif event['type'] == 'program change':
            hexstring = event['type hex'] + event['channel'] + event['program']

        elif event['type'] == 'note on' or event['type'] == 'note off':
            hexstring = event['type hex'] + event['channel'] + event['note'] + event['velocity']

        return hexstring
    
class Music:
    def __init__(self, name):
        self._parse_midi(name)
        self.name = name
    
    def _parse_midi(self, name):
        midi_file = f'./{name}.mid'

        with open(midi_file, 'rb') as f:
            hexdata = binascii.hexlify(f.read())
        hexdata_str = hexdata.decode("utf-8")

        MTrk = '4d54726b'
        tracks_str = []
        for i,chunk in enumerate(hexdata_str.split(MTrk)):
            if i == 0:
                header_str = chunk
            else:
                tracks_str.append (MTrk + chunk)

        track_objs = [Track(t) for t in tracks_str]
        
        self.header_str = header_str
        self.track_objs = track_objs

    def keyshift(self, shift_amount):
        track_objs = self.track_objs.copy()
        for obj in track_objs:
            for event in obj.segs:
                if event['type'] == 'note on' or event['type'] == 'note off':
                    note_hex = '0x'+event['note']
                    note_integer = int(note_hex, 16)
                    note_integer_shifted = note_integer + shift_amount
                    note_hex_shifted = hex(note_integer_shifted)[2:]
                    note_hex_shifted = note_hex_shifted.rjust(2, '0')
                    event['note'] = note_hex_shifted
        self.track_objs = track_objs
    
    def export_midi(self, name):
        recovered_tracks = [T.segs_to_hexstring() for T in self.track_objs]
        with open(f'{name}.mid', 'wb') as fout:
            fout.write( binascii.unhexlify(self.header_str + ''.join(recovered_tracks)) )

In [13]:
midi_file = 'ryuichi3.mid'

with open(midi_file, 'rb') as f:
    hexdata = binascii.hexlify(f.read())
hexdata_str = hexdata.decode("utf-8")

MTrk = '4d54726b'
tracks_str = []
for i,chunk in enumerate(hexdata_str.split(MTrk)):
    if i == 0:
        header_str = chunk
    else:
        tracks_str.append (MTrk + chunk)

In [14]:
def convert_hex_to_arr(hexstring):
    '''
    last element of the felt array is an integer indicating the hex-length of the last felt value
    '''
    arr = []
    s = hexstring
    while( len(s)>62 ):
        felt_hex = s[0:62]
        arr.append( int(felt_hex, 16) )
        s = s[62:]
    felt_hex = s
    last_length = len(felt_hex)
    arr.append( int(felt_hex, 16) )
    arr.append(last_length)

    return arr

def generate_single(hexstring, name):
    ret = []

    ret.append( '@view')
    ret.append(f'func {name} {{')
    ret.append( '        range_check_ptr')
    ret.append( '    } () -> (')
    ret.append( '        z_len : felt,')
    ret.append( '        z : felt*')
    ret.append( '    ):')
    ret.append( '    alloc_locals\n')
    ret.append( '    let (local z) = alloc()\n')

    arr = convert_hex_to_arr(hexstring)
    for i,num in enumerate(arr):
        ret.append(f'    assert [z+{i}] = {num}')
    ret.append(f'    let z_len = {i+1}')
    ret.append('\n    return (z_len, z)')
    ret.append('end')

    for line in ret:
        print(line)

In [15]:
generate_single(hexdata_str, 'music')

@view
func music {
        range_check_ptr
    } () -> (
        z_len : felt,
        z : felt*
    ):
    alloc_locals

    let (local z) = alloc()

    assert [z+0] = 136629785046748551405435688290012123131790238700113534911158541394819153920
    assert [z+1] = 1762318216636338976432867993573267631005408414846004194592588298130649972
    assert [z+2] = 193258903252201988734951255801966681645259054397445142402996957317110524769
    assert [z+3] = 193355562949225473421522926162276094281234580689804105518375139058066222703
    assert [z+4] = 1656641865430759840151478342257009721025894147065113368679749484121440324
    assert [z+5] = 34786534616229639124782824949843016706926835441768096785993085368324345871
    assert [z+6] = 311406805020597082985528550051478436489035998565030200852676864222793062703
    assert [z+7] = 4750136620008811638027811096291745619598619044706562866113449395756226816
    assert [z+8] = 22198557174187945661722937401729705167449238483722563910878227075045546295
  